In [32]:
from nytimesarticle import articleAPI
api = articleAPI('your key here')

In [56]:
articles = api.search( q = 'Amnesty',fq = {'headline':'Amnesty', 'source':['Reuters','AP', 'The New York Times']}, begin_date = 20170310 )

In [57]:
articles

{u'copyright': u'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 u'response': {u'docs': [{u'_id': u'58ec0e8a95d0e02489b964fc',
    u'abstract': None,
    u'blog': [],
    u'byline': {u'original': u'By CHRIS BUCKLEY',
     u'person': [{u'firstname': u'Chris',
       u'lastname': u'BUCKLEY',
       u'organization': u'',
       u'rank': 1,
       u'role': u'reported'}]},
    u'document_type': u'article',
    u'headline': {u'main': u'China\u2019s Death Penalty Is Still Veiled in Secrecy, Amnesty Says',
     u'print_headline': u'China\u2019s Death Penalty Is Still Veiled in Secrecy, Amnesty International Report Says'},
    u'keywords': [{u'isMajor': u'N',
      u'name': u'organizations',
      u'rank': 1,
      u'value': u'Amnesty International'},
     {u'isMajor': u'N',
      u'name': u'organizations',
      u'rank': 2,
      u'value': u'Communist Party of China'},
     {u'isMajor': u'N',
      u'name': u'glocations',
      u'rank': 3,
      u'value': u'China'},
   

In [58]:
#This function takes in a response to the NYT api and parses the articles into a list of dictionaries

def parse_articles(articles):
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        if i['abstract'] is not None:
            dic['abstract'] = i['abstract'].encode("utf8")
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['desk'] = i['news_desk']
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        dic['section'] = i['section_name']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['source'] = i['source']
        dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects
        news.append(dic)
    return(news)

In [59]:
#This function accepts a year in string format (e.g.'1980') and a query (e.g.'Amnesty International') and it will 
#return a list of parsed articles (in dictionaries) for that year.

def get_articles(date,query):
    all_articles = []
    for i in range(0,100): #NYT limits pager to first 100 pages. But rarely will you find over 100 pages of results anyway.
        articles = api.search(q = query, fq = {'source':['Reuters','AP', 'The New York Times']},\
                              begin_date = date + '0101', end_date = date + '1231', sort='oldest',page = str(i))
        articles = parse_articles(articles)
        all_articles = all_articles + articles
    return(all_articles)

In [63]:
#This function will input a year and search query, and return a list of all articles that fit those parameters, 
#parsing them into a nice list of dictionaries.

Amnesty_all = []
for i in range(2016,2017):
    print 'Processing' + str(i) + '...'
    Amnesty_year = get_articles(str(i),'Amnesty International')
    Amnesty_all = Amnesty_all + Amnesty_year

Processing2016...


KeyError: 'response'

In [ ]:
#To export into a csv

import csv
keys = Amnesty_all[0].keys()
with open('amnesty-mentions.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(Amnesty_all)